# Image Denoising: Tikhonov and Total Variation Regularization

The problem of removing noise from an image without blurring
sharp edges can be formulated as an infinite-dimensional minimization
problem. Given a possibly noisy image $u_0(x,y)$ defined within a
square domain $\Omega$, we would like to
find the image $u(x,y)$ that is closest in the $L_2$ sense, i.e. we
want to minimize 

$$ \mathcal{F}_{LS} := \frac{1}{2}\int_\Omega (u - u_0)^2 \; d\boldsymbol{x}, $$


while also removing noise, which is assumed to comprise very ``rough''
components of the image. This latter goal can be incorporated as an
additional term in the objective, in the form of a penalty, i.e., 

$$ \mathcal{R}_{TN} := \! \frac{\alpha}{2}\int_\Omega \nabla u
\cdot \! \nabla u \; d\boldsymbol{x}, $$
where $\alpha$ acts as a ``diffusion'' coefficient that controls
how strongly we impose the penalty, i.e.\ how much smoothing
occurs. Unfortunately, if there are sharp edges in the image, this
so-called {\em Tikhonov (TN) regularization} will blur them. Instead,
in these cases we prefer the so-called {\em total variation (TV)
regularization},

$$ \mathcal{R}_{TV} := \! \alpha\int_\Omega (\nabla u \cdot \! \nabla
u)^{\frac{1}{2}} \; d\boldsymbol{x} $$

where (we will see that) taking the square root is the key to
preserving edges. Since 
$\mathcal{R}_{TV}$ is not differentiable when $\nabla u =
\boldsymbol{0}$, it is usually modified to include a positive parameter
$\varepsilon$ as follows:

$$ \mathcal{R}^{\varepsilon}_{TV} := \!  \alpha \int_\Omega (\nabla u \cdot
\! \nabla u + \varepsilon)^{\frac{1}{2}} \; d\boldsymbol{x}. $$
We wish to study the performance of the two denoising functionals
$\mathcal{F}_{TN}$ and $\mathcal{F}^{\varepsilon}_{TV}$, where

$$ \mathcal{F}_{TN}  := \mathcal{F}_{LS} + \mathcal{R}_{TN} $$
and

$$ \mathcal{F}^{\varepsilon}_{TV}  := \mathcal{F}_{LS} + \mathcal{R}^{\varepsilon}_{TV}. $$

We will prescribe the homogeneous Neumann condition 
$\nabla u \cdot \boldsymbol{n}=0$ on the four sides of the square,
which amounts to assuming that the image intensity does not change
normal to the boundary of the image. 

## 1. Setup

- We generate a rectangular mesh of size `Lx` by `Ly`.

- We define a linear finite element space.

- We generate two finite element functions `u_true` and `u_0` that represent the true image and the noisy image respectively.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from dolfin import *

import math
import numpy as np
import logging



from unconstrainedMinimization import InexactNewtonCG

logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
set_log_active(False)

data = np.loadtxt('image.dat', delimiter=',')
np.random.seed(seed=1)
noise_std_dev = .3
noise = noise_std_dev*np.random.randn(data.shape[0], data.shape[1])
Lx = float(data.shape[1])/float(data.shape[0])
Ly = 1.

class Image(Expression):
    def __init__(self, Lx, Ly, data, **kwargs):
        self.data = data
        self.hx = Lx/float(data.shape[1]-1)
        self.hy = Ly/float(data.shape[0]-1)
        
    def eval(self, values, x):
        j = int(math.floor(x[0]/self.hx))
        i = int(math.floor(x[1]/self.hy))
        values[0] = self.data[i,j]
        
mesh = RectangleMesh(Point(0,0),Point(Lx,Ly),200, 100)
V = FunctionSpace(mesh, "Lagrange",1)
trueImage = Image(Lx,Ly,data,degree = 1)
noisyImage = Image(Lx,Ly,data+noise, degree = 1)
u_true  = interpolate(trueImage, V)
u_0 = interpolate(noisyImage, V)


plt.figure(figsize=[12,24])
plt.subplot(1,2,1)
plot(u_true, title="True Image")
plt.subplot(1,2,2)
plot(u_0, title="Noisy Image")
plt.show()


## 2. The misfit functional and the true error functional

Here we describe the variational forms for the $L_2$ misfit functional
$$ \mathcal{F}_{LS} := \frac{1}{2}\int_\Omega (u - u_0)^2 \; d\boldsymbol{x}, $$
and its first and second variations.

Since we also know the true image `u_true` (*this will not be the case for real applications*)
we can also write the true $L_2$ error functional 
$$ \mathcal{F}_{\rm true} := \frac{1}{2}\int_\Omega (u - u_{\rm true})^2 \; d\boldsymbol{x}. $$

Finally, we define some values of $\alpha$ ($\alpha = 1, 10^{-1}, 10^{-2}, 10^{-3}, 10^{-4}$) for the choice of the regularization paramenter.

In [ ]:
u = Function(V)
u_hat = TestFunction(V)
u_tilde = TrialFunction(V)

F_ls = Constant(.5)*inner(u - u_0, u - u_0)*dx
grad_ls = inner(u - u_0, u_hat)*dx
H_ls = inner(u_tilde, u_hat)*dx

trueError = inner(u - u_true, u - u_true)*dx

n_alphas = 5
alphas = np.power(10., -np.arange(n_alphas))

## 3. Tikhonov regularization

The function `TNsolution` computes the solution of the denoising inverse problem using Tikhonov regularization for a given amount a regularization $\alpha$.

More specifically it minimizes the functional
$$ \mathcal{F}_{TN}(u) = \frac{1}{2}\int_\Omega (u - u_0)^2 \; d\boldsymbol{x} + \frac{\alpha}{2}\int_\Omega \nabla u \cdot \nabla u d\boldsymbol{x}. $$

The first variation of $\mathcal{F}_{TN}$ reads
$$ \delta_u \mathcal{F}_{TN}(u, \hat{u}) = \int_\Omega (u - u_0) \hat{u} \; d\boldsymbol{x} + \alpha \int_\Omega \nabla u \cdot \nabla \hat{u}  d\boldsymbol{x}, $$
and the second variation is
$$ \delta_u^2 \mathcal{F}_{TN}(u, \hat{u}, \tilde{u}) = \int_\Omega \tilde{u}\, \hat{u} \; d\boldsymbol{x} + \alpha \int_\Omega \nabla \tilde{u} \cdot \nabla \hat{u}\, d\boldsymbol{x}. $$

The best reconstruction of the original image is obtained for $\alpha = 10^{-3}$, however we notice that the sharp edges of the image are lost in the reconstruction.

In [ ]:
def TNsolution(alpha):
    R_tn = Constant(.5)*inner(nabla_grad(u), nabla_grad(u))*dx
    grad_tn = inner(nabla_grad(u), nabla_grad(u_hat))*dx
    H_tn = inner(nabla_grad(u_tilde), nabla_grad(u_hat))*dx
    F = F_ls + alpha*R_tn
    grad = grad_ls + alpha*grad_tn
    H = H_ls + alpha*H_tn
    solve(H==-grad, u)
    print "{0:15e} {1:15e} {2:15e} {3:15e} {4:15e}".format(
           alphas[i], assemble(F), assemble(F_ls), assemble(alpha*R_tn), assemble(trueError))
    plt.figure()
    plot(u)
    plt.show()

print "{0:15} {1:15} {2:15} {3:15} {4:15}".format("alpha", "F", "F_ls", "alpha*R_tn", "True Error")
for i in range(n_alphas):
    TNsolution(Constant(alphas[i]))

## 4. Total Variation regularization

The function `TVsolution` computes the solution of the denoising inverse problem using Total Variation regularization for a given amount a regularization $\alpha$ and perturbation $\varepsilon$.

More specifically it minimizes the functional
$$ \mathcal{F}_{TV}(u) = \frac{1}{2}\int_\Omega (u - u_0)^2 \; d\boldsymbol{x} + \frac{\alpha}{2}\int_\Omega \left( \nabla u \cdot \nabla u + \varepsilon \right)^{\frac{1}{2}} d\boldsymbol{x}. $$

The first variation of $\mathcal{F}_{TV}$ reads
$$ \delta_u \mathcal{F}_{TV}(u, \hat{u}) = \int_\Omega (u - u_0)\hat{u}  \; d\boldsymbol{x} + \alpha \int_\Omega \frac{1}{\left( \nabla u \cdot \nabla u + \varepsilon \right)^{\frac{1}{2}}}\nabla u \cdot \nabla \hat{u}  d\boldsymbol{x}, $$
and the second variation is
$$ \delta_u^2 \mathcal{F}_{TV}(u, \hat{u}, \tilde{u}) = \int_\Omega \tilde{u} \hat{u} \; d\boldsymbol{x} + \alpha \int_\Omega \frac{1}{\left( \nabla u \cdot \nabla u + \varepsilon \right)^{\frac{1}{2}}} \left[ \left( I - \frac{\nabla u \otimes \nabla u}{\nabla u \cdot \nabla u + \varepsilon}\right) \nabla \tilde{u}\right] \cdot \nabla \hat{u} d\boldsymbol{x}. $$

The highly nonlinear term $\left( I - \frac{\nabla u \otimes \nabla u}{\nabla u \cdot \nabla u + \varepsilon}\right) $ in the second variation poses a substantial challange for the convergence of the Newton's method. In fact, the converge radius of the Newtos's method is extremely small.
For this reason in the following we will replace the second variation with the variational form
$$ H_{\rm approx} = \int_\Omega \tilde{u}\,\hat{u} \; d\boldsymbol{x} + \alpha \int_\Omega \frac{1}{\left( \nabla u \cdot \nabla u + \varepsilon \right)^{\frac{1}{2}}} \tilde{u} \cdot \nabla \hat{u} d\boldsymbol{x}. $$

The resulting method will exhibit only a first order convergence rate but it will be more robust for small values of $\varepsilon$.

The best reconstruction of the original image is obtained for $\alpha = 10^{-2}$. We also notice that Total Variation does a much better job that Tikhonov regularization in preserving the sharp edges of the original image.

In [ ]:
def TVsolution(eps, alpha):
    def TV(u, eps):
        return sqrt( inner(nabla_grad(u), nabla_grad(u)) + eps)
    
    def scaled_grad(u, eps):
        return nabla_grad(u)/TV(u,eps)
    
    R_tv = TV(u, eps)*dx
    F = F_ls + alpha*R_tv
    grad_tv = Constant(1.)/TV(u, eps)*inner(nabla_grad(u), nabla_grad(u_hat))*dx
    grad = grad_ls + alpha*grad_tv
    H_tv = Constant(1.)/TV(u, eps)*inner(nabla_grad(u_tilde), nabla_grad(u_hat))*dx #+ \
    #       Constant(1.)/TV(u, eps)*inner(outer(scaled_grad(u,eps), scaled_grad(u,eps) )*nabla_grad(u_tilde), nabla_grad(u_hat))*dx 
    H = H_ls + alpha*H_tv
    solver = InexactNewtonCG()
    solver.parameters["rel_tolerance"] = 1e-5
    solver.parameters["abs_tolerance"] = 1e-6
    solver.parameters["gdu_tolerance"] = 1e-18
    solver.parameters["max_iter"] = 1000
    solver.parameters["c_armijo"] = 1e-5
    solver.parameters["print_level"] = -1
    solver.parameters["max_backtracking_iter"] = 10
    solver.solve(F, u, grad, H)

    print "{0:15e} {1:5} {2:4d} {3:15e} {4:15e} {5:15e} {6:15e}".format(
           alphas[i], solver.converged, solver.it, assemble(F), assemble(F_ls), assemble(alpha*R_tv), assemble(trueError))
    
    plt.figure()
    plot(u)
    plt.show()
    
print "{0:15} {1:5} {2:4} {3:15} {4:15} {5:15} {6:15}".format("alpha", "converged", "nit", "F", "F_ls", "alpha*R_tn", "True Error")

eps = Constant(1e-4)
for i in range(n_alphas):
    TVsolution(eps, Constant(alphas[i]))

Copyright (c) 2016, The University of Texas at Austin & University of California, Merced.

All Rights reserved.
See file COPYRIGHT for details.

This file is part of the hIPPYlib library. For more information and source code availability see https://hippylib.github.io.

hIPPYlib is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License (as published by the Free Software Foundation) version 2.0 dated June 1991.